# 데이터 가져오기

In [1]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/20143232/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
import urllib.request
import zipfile
from lxml import etree
import re
from nltk.tokenize import word_tokenize, sent_tokenize

In [3]:
# 데이터 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/GaoleMeng/RNN-and-FFNN-textClassification/master/ted_en-20160408.xml", 
                           filename="ted_en-20160408.xml")

('ted_en-20160408.xml', <http.client.HTTPMessage at 0x7f19d228a978>)

In [4]:
targetXML=open('ted_en-20160408.xml', 'r', encoding='UTF8') # xml 파일 오픈
target_text = etree.parse(targetXML) # 오픈한 xml파일을 가져오기


# xml 파일로부터 <content>와 </content> 사이의 내용만 가져온다.
parse_text = '\n'.join(target_text.xpath('//content/text()'))

# 정규 표현식의 sub 모듈을 통해 content 중간에 등장하는 (Audio), (Laughter) 등의 배경음 부분을 제거.
# 해당 코드는 괄호로 구성된 내용을 제거.
content_text = re.sub(r'\([^)]*\)', '', parse_text)

# 입력 코퍼스에 대해서 NLTK를 이용하여 문장 토큰화를 수행.
sent_text = sent_tokenize(content_text)

# 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환.
normalized_text = []
for string in sent_text:
    tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
    normalized_text.append(tokens)

# 각 문장에 대해서 NLTK를 이용하여 단어 토큰화를 수행.
result = [word_tokenize(sentence) for sentence in normalized_text]

# 글로브 훈련시키기

In [5]:
from glove import Corpus, Glove

corpus = Corpus() 
corpus.fit(result, window=5)
# 훈련 데이터로부터 GloVe에서 사용할 동시 등장 행렬 생성

glove = Glove(no_components=100, learning_rate=0.05)
glove.fit(corpus.matrix, epochs=20, no_threads=4, verbose=True)
glove.add_dictionary(corpus.dictionary)
# 학습에 이용할 쓰레드의 개수는 4로 설정, 에포크는 20.

Performing 20 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19


In [6]:
model_result1=glove.most_similar("man")
print(model_result1)

[('woman', 0.9643548337891651), ('guy', 0.8911840759713822), ('girl', 0.8608303561601449), ('young', 0.8448245751406213)]


In [7]:
model_result1=glove.most_similar("boy")
print(model_result1)

[('girl', 0.9453741595581819), ('woman', 0.8491058372989381), ('kid', 0.8452228951943835), ('man', 0.8294323190857581)]


In [8]:
model_result1=glove.most_similar("university")
print(model_result1)

[('harvard', 0.8900605504930895), ('mit', 0.8575071156739342), ('cambridge', 0.843313592752099), ('stanford', 0.8349776524106634)]
